# Problem Setup

Given a triangle mesh as a set of vertices $V\in \mathbb R^{n\times 3} $ and a set of faces $F\in\mathbb R^{m\times 3}$. We want to output a deformed shape $\tilde V$. The output shape will have each sub-component in the style of axis-aligned cubes and will remain the geometric details of the original mesh. 

# Method

The problem is view as an energy optimization problem. We want to minimze the ARAP energy with a l1 regularization. Denote the edges at rest state as $d_{ij} = v_j-v_i$ and at deformed states as $\tilde d_{ij} = \tilde v_j - \tilde v_i$, we can write the optimization problem as 
\begin{align*}
\text{minimize} \quad&\sum_{i\in V}\sum_{j\in \mathcal N(i)} \frac{w_{ij}}{2}\|R_id_{ij} - \tilde d_{ij}\|_F^2 + \lambda a_i \|R_i\tilde n_i\|_1\\
\text{subject to} \quad& \tilde V, R_1,..., R_n
\end{align*}
The first term is the <a href="https://github.com/alecjacobson/geometry-processing-deformation/blob/master/papers/GeomElastic.pdf">ARAP energy</a> where $R_i$ is the rotation matrix and $w_{ij}$ is the cotangent weight and $\mathcal N(i)$ is the "spokes and rims" edges of the $i$th vertex.  
The second term is the new, added L1 regularization. $\hat n_i$ denotes the area-weighted unit normal vector of $v_i$ and $a_i$ is the barycentric area of $v_i$ and $\lambda$ is a tuning parameter. 

# Local Step
The local step involves finding the rotation matrix $R_1,..., R_n$, i.e. For each vertex $i$, we are to optimize 
$$R_i^* = \arg\min_{R_i\in SO(3)}\sum_{j\in \mathcal N(i)} \frac{w_{ij}}{2}\|R_id_{ij} - \tilde d_{ij}\|_F^2 + \lambda a_i \|R_i\tilde n_i\|_1$$
note that the ARAP energy can be expressed in a matrix formations
$$\frac12 (R_iD_i-\tilde D_i)^T W_i (R_iD_i-\tilde D_i)$$
where $D_i,\tilde D_i \in \mathbb R^{3\times |\mathcal N(i)|}$ are stacked rim/spoke edge vectors and $W_i$ is the diagonal matrix of $w_1,...,w_n$. Then, write $z = R_i\hat n_i$, we can turn the formation into
\begin{align*}
\text{minimize}_{z_, R_i} \quad &\frac12 (R_iD_i-\tilde D_i)^T W_i (R_iD_i-\tilde D_i)+\lambda a_i\|z\|_1\\
\text{subject to} \quad &z-R_i\hat n_i = 0
\end{align*}
We can solve the local step using the scaled-form <a href="https://web.stanford.edu/~boyd/papers/pdf/admm_slides.pdf">ADMM</a> updates. 

Applying ADMM, the update steps are 
\begin{align*}
R_i^{k+1} &= \arg\min \frac12(R_iD_i-\tilde D_i)^T W_i (R_iD_i-\tilde D_i) +\frac{\rho^k}2\|R_i\hat n_i - z^k + u^k\|_2^2\\
z^{k+1} &= \arg\min \lambda a_i \|z\|_1 + \frac{\rho^k}2\|R_i^{k+1} \hat n_i - z + u^k\|_2^2\\
\tilde u^{k+1} &= u^k + R_i^{k+1} \hat n_i - z^{k+1}\\
\rho^{k+1}, u^{k+1} &= update(\rho^k)
\end{align*}

Then, consider each update, 

The rotation update can be viewed as 
\begin{align*}
R_i^{k+1} &= \arg\max tr(R_i M_i)\\
M_i &= \begin{bmatrix}[D_i]&[\hat n_i]\end{bmatrix}
\begin{bmatrix}[W_i]&0\\0&\rho^k\end{bmatrix}
\begin{bmatrix}[\tilde D_i]\\ [(z^k-u^k)^T] \end{bmatrix}
\end{align*}
This becomes an Orthogonal Procrustes problem, and the solution given by $M = U\Sigma V^T$ through single value decomposition, and then $R = UV^T$ up to $det(R) > 0$ by alternating the sign of $U$'s column.  

